In [ ]:
!pip install transformers
!pip install PyPDF

In [ ]:
import pandas as pd
import PyPDF2
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


# --- File Extraction Functions ---

In [ ]:
def extract_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        text = " ".join([reader.getPage(i).extractText() for i in range(reader.numPages)])
    return text

In [ ]:
def extract_words_from_csv(file_path):
    df = pd.read_csv(file_path)
    df = df.astype(str)
    # Splitting each row's entry by spaces to get individual words
    return [entry.split() for entry in df.values.flatten()]

In [ ]:
def extract_from_txt(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [ ]:
def extract_data(file_path, file_format):
    if file_format == 'pdf':
        return extract_from_pdf(file_path)
    elif file_format == 'csv':
        return extract_words_from_csv(file_path)
    elif file_format == 'txt':
        return extract_from_txt(file_path)
    else:
        raise ValueError("Unsupported file format")

# --- LLM Functions ---

In [ ]:
def text_to_embedding(text):
    # Set the padding token if it's not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Tokenize the text with a higher max_length and generate the attention mask
    tokens = tokenizer.encode_plus(text, max_length=512, truncation=True, padding='max_length', return_tensors="pt")
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Use the embedding of the [CLS] token
    return outputs[0][:, 0, :]



In [ ]:
def cosine_similarity(a, b):
    return torch.nn.functional.cosine_similarity(a, b)

In [ ]:
policy_data = extract_data('policies_questions.csv', 'csv')
logs_data = extract_data('logs.csv', 'csv')

In [ ]:
def extract_relevant_policy_by_embedding(log_embedding, policy_data):
    # We assume that policy_data is a list of individual policies
    best_score = -float('inf')
    best_policy = ""

    for policy in policy_data:

        policy_embedding = text_to_embedding(policy)
        similarity_score = cosine_similarity(log_embedding, policy_embedding)

        if similarity_score > best_score:
            best_score = similarity_score
            best_policy = policy

    return best_policy

In [ ]:
remediation_statements = {}
for log in non_compliant_logs:
    remediation_statements[log] = generate_remediation(log)
print(remediation_statements)

In [ ]:
policy_data_list = extract_data('policies_questions.csv','csv')
logs_list = extract_data('logs.csv' ,'csv')
# Drop rows containing 'nan' values
policy_data_list = [policy_data for policy_data in policy_data_list if policy_data != ['nan']]
logs_list = [log_data for log_data in logs_list if log_data != ['nan']]
print(policy_data_list)
print(logs_list)

In [ ]:
policy_data_list = extract_data('policies_questions.csv','csv')
logs_list = extract_data('logs.csv' ,'csv')
# Drop rows containing 'nan' values
policy_data_list = [policy_data for policy_data in policy_data_list if policy_data != ['nan']]
logs_list = [log_data for log_data in logs_list if log_data != ['nan']]
print(policy_data_list)
print(logs_list)
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
import torch.nn.functional as F
import pandas as pd

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name, num_labels=2)
model = BertForSequenceClassification.from_pretrained(model_name, config=config)


# Create an empty list to store results
results = []
i = 0

joined_logs_list = []
for log in logs_list:
  one_log = ' '.join(log)
  joined_logs_list.append(one_log)

joined_policy_list = []
for pol in policy_data_list:
  one_pol = ' '.join(pol)
  joined_policy_list.append(one_pol)

for log_data in joined_logs_list:
  log_embedding = text_to_embedding(log)
  relevant_policy = extract_relevant_policy_by_embedding(log_embedding, joined_policy_list)
  question = "Is this compliant with the policy?"
  for policy_data in relevant_policy:

        # Combine policy and log as text
        text = f"Policy: {policy}\nLog: {log}\nQuestion: {question}"
        # print(policy)
        # print(log)
        i += 1
        print(i)
        # Tokenize and classify
        inputs = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        outputs = model(**inputs)
        logits = outputs.logits

        # Apply softmax to get probabilities
        probs = F.softmax(logits, dim=1)

        # Get the predicted label and confidence score
        predicted_label = torch.argmax(probs).item()
        confidence_score = probs[0][predicted_label].item()

        # Map predicted label to "Yes" or "No"
        answer = "Yes" if predicted_label == 1 else "No"

        # Append the result to the list
        results.append({"Policy": policy, "Log": log, "Answer": answer, "Confidence Score": confidence_score})

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

# Print the results DataFrame
print(results_df)


In [ ]:
excel_file_path = 'results.xlsx'
results_df.to_excel(excel_file_path, index=False)

In [ ]:
pip install optuna

In [ ]:
import pandas as pd
# Read the CSV file
csv_file_path = 'labeled.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
out_1 = df[df.Label == 1]
out_0 = df[df.Label == 0]

out_0 = out_0.sample(n=240,replace = True)

df = pd.concat([out_1, out_0], axis=0).sample(frac=1).reset_index(drop=True)


In [ ]:
# Extract data from specific columns into lists
policy_list = df['Policy'].tolist()
log_list = df['Log'].tolist()
labels_list = df['Label'].tolist()

In [ ]:
text_list = []
questions = []

for policy_data in policy_list:
    # Remove extra spaces within policy_data
    cleaned_policy_data = ' '.join(policy_data.split())  # This will remove extra spaces
    policy = cleaned_policy_data

    for log_data in log_list:
        cleaned_log = ' '.join(log_data.split())  # Join the words to form the log text
        log = cleaned_log
        question = 'Is the log compliant with the policy?'

        # Combine policy and log as text
        combined_text = f"Policy: {policy}\nLog: {log}"
        text_list.append(combined_text)  # Append the combined text to the list
        questions.append(question)
print(len(text_list))
print(len(questions))

In [ ]:
from tqdm import tqdm

In [ ]:
df.tail()

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name, num_labels=2)

labels = labels_list
# Combine policy and log as text
policy_text = ' '.join(policy_list[0])  # Join the words to form the policy text

# Split data into train and validation sets
log_texts_train, log_texts_val, labels_train, labels_val = train_test_split(log_list[:], labels[:], test_size=0.2, random_state=42)

log_texts_train =log_texts_train + log_texts_val
labels_train = labels_train + labels_val



In [ ]:
labels_train


In [ ]:
# Grid of hyperparameters
learning_rates = [1e-4]
batch_sizes = [40]

# Hyperparameter tuning loop
best_accuracy = 0.0
best_hyperparameters = {}

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        model = BertForSequenceClassification.from_pretrained(model_name, config=config)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        loss_fn = torch.nn.CrossEntropyLoss()

        # Create data loaders
        train_inputs = tokenizer(log_texts_train, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt')
        val_inputs = tokenizer(log_texts_val, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt')

        train_labels = torch.tensor(labels_train)
        val_labels = torch.tensor(labels_val)

        train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
        val_data = torch.utils.data.TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_labels)

        train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

        # Training loop
        num_epochs = 20
        for epoch in tqdm(range(num_epochs)):
            # model = model.to('cuda')
            model.train()
            for batch in train_loader:
                optimizer.zero_grad()
                input_ids, attention_mask, labels = batch
                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                loss = loss_fn(logits, labels)
                loss.backward()
                optimizer.step()
                print(loss)

            # Validation
            model.eval()
            correct_predictions = 0
            total_samples = 0
            with torch.no_grad():
                for batch in val_loader:
                    input_ids, attention_mask, labels = batch
                    outputs = model(input_ids, attention_mask=attention_mask)
                    logits = outputs.logits
                    _, predicted = torch.max(logits, dim=1)
                    print(predicted,labels)
                    correct_predictions += (predicted == labels).sum().item()
                    total_samples += labels.size(0)

            accuracy = correct_predictions / total_samples
            print(f"Learning Rate: {learning_rate} - Batch Size: {batch_size} - Accuracy: {accuracy:.4f}")

            # Keep track of best hyperparameters
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_hyperparameters = {"Learning Rate": learning_rate, "Batch Size": batch_size}

print("Best Hyperparameters:", best_hyperparameters)

In [ ]:
best_model_dir = "best_bert_model"
model.save_pretrained(best_model_dir)

In [ ]:
newpolicy_data_list = extract_data('policies_questions.csv','csv')
newlogs_list = extract_data('logs.csv' ,'csv')

# Drop rows containing 'nan' values
newpolicy_data_list = [policy_data for policy_data in newpolicy_data_list if policy_data != ['nan']]
newlogs_list = [log_data for log_data in newlogs_list if log_data != ['nan']]
print(newpolicy_data_list)
print(newlogs_list)


In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)
best_model_dir = "best_bert_model"  # Change this to the directory where you saved your best model
loaded_model = BertForSequenceClassification.from_pretrained(best_model_dir, config=config)

# Assuming policy_list and log_list contain your data
question = 'Is the log compliant with the policy?'
results = []  # To store the results
i = 0
# Loop through policy and log pairs
for policy in newpolicy_data_list:
  for log in  newlogs_list:
    input_text = f"Policy: {policy}\nLog: {log}\nQuestion: {question}\n"
    inputs = tokenizer(input_text, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt')
    i+=1
    print(i)

    with torch.no_grad():
        outputs = loaded_model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    results.append({"Policy": policy, "Log": log, "Predicted_Label": predicted_label})

# Create a DataFrame from the results
output_df = pd.DataFrame(results)

# Display the DataFrame
print(output_df)


In [ ]:
excel_file_path = 'results.xlsx'
output_df.to_excel(excel_file_path, index=False)

In [ ]:
best_model_dir = "best_bert_model"
model.save_pretrained(best_model_dir)


In [ ]:
import shutil
shutil.make_archive("best_bert_model", "zip", "best_bert_model")

In [ ]:
from google.colab import files
files.download("best_bert_model.zip")

In [ ]:
# Model Declaration
model = BertForSequenceClassification.from_pretrained(model_name, config=config)

# hyperparameters
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
learning_rates = 1e-4
batch_sizes = 40
num_epochs = 20

# Create data loaders
train_inputs = tokenizer(log_texts_train, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt')
val_inputs = tokenizer(log_texts_val, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt')

train_labels = torch.tensor(labels_train)
val_labels = torch.tensor(labels_val)

train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
val_data = torch.utils.data.TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

# Training loop
for epoch in tqdm(range(num_epochs)):
    # model = model.to('cuda')
    model.train()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        print(loss)

    # Validation
    model.eval()
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted = torch.max(logits, dim=1)
            print(predicted,labels)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = correct_predictions / total_samples
    print(f"Learning Rate: {learning_rate} - Batch Size: {batch_size} - Accuracy: {accuracy:.4f}")


